<a href="https://colab.research.google.com/github/lucasreis95/world-surf-league-data/blob/main/notebooks/etl/08_gold_events_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libs
import pandas as pd
import pandas_gbq
import numpy as np
import re

In [ ]:
# read df from gbq
df_ranking = pandas_gbq.read_gbq(
                             query_or_table = 'wsl-data-397017.02_silver.wsl_ranking_yearly'
                             )

Downloading: 100%|██████████|


In [ ]:
# read df from gbq
df_events = pandas_gbq.read_gbq(
                             query_or_table = 'wsl-data-397017.02_silver.wsl_events_dimensions'
                             )

Downloading: 100%|██████████|


In [ ]:
# read df from gbq
df_injuries = pandas_gbq.read_gbq(
                             query_or_table = 'wsl-data-397017.02_silver.wsl_atheletes_events_injuries'
                             )

Downloading: 100%|██████████|


In [ ]:
df_events_results = df_ranking.drop(columns = ['athlete_country', 'rank_position', 'regular_season_total_points', 'total_points'])
df_events_results = pd.melt(df_events_results, id_vars = ['athlete_name', 'season_year'])
df_events_results = df_events_results.rename(columns = {'variable':'event_order', 'value':'points_earned'})
# keep just number on event order
df_events_results['event_order'] = df_events_results['event_order'].str.extract('(\d+)')
df_events_results['event_order'] = df_events_results['event_order'].astype(int)
# drop events with no points earned
df_events_results = df_events_results[df_events_results['points_earned'].notnull()]
# merge event information on df
df_events_results = df_events_results.merge(df_events, on = ['season_year', 'event_order'], how = 'left')

## Update points earned between years
Format of how points are distruited changed few times on wsl history since 2010, we can found their values on wsl rule book of respective years.

2022: https://www.worldsurfleague.com/asset/33448/2022+WSL+Rule+Book+-+April+2022+-+Final+Form.pdf

2018: https://www.worldsurfleague.com/asset/18359/2018+WSL+Rule+Book+-+03092018.pdf

2017: https://www.worldsurfleague.com/asset/11489/2017+WSL+Rule+Book+-+01092017.pdf


In [ ]:
score_to_replace_dict = {
    8000:7800, #2 place
    6500:6085, #3 place
    5200:4745, #5 place
    5250:4745, #5 place
    3700:3320, #9 place
    4000:3320, #9 place
    3750:3320, #9 place
    1665:1330, #13 place (past)/#17 place(actual)
    1750:1330, #13 place (past)/#17 place(actual)
    420:265, #25 place (past)/#33 place(actual)
    500:265, #25 place (past)/#33 place(actual)
    395:265, #25 place (past)/#33 place(actual)
}

In [ ]:
df_events_results['points_earned_update'] = df_events_results['points_earned'].replace(score_to_replace_dict)

In [ ]:
# add injuried information
df_inj_to_merge = df_injuries.copy()
# create auxiliar colummn
df_inj_to_merge['injuried_flag'] = True
# convert to string to merge
df_events_results['season_year'] = df_events_results['season_year'].astype(str)
df_events_results['event_order'] = df_events_results['event_order'].astype(str)
df_inj_to_merge['season_year'] = df_inj_to_merge['season_year'].astype(str)
df_inj_to_merge['event_order'] = df_inj_to_merge['event_order'].astype(str)
#
df_events_results = df_events_results.merge(df_inj_to_merge, on = ['athlete_name', 'season_year', 'event_order'], how = 'left')
# replace event point earned to np.nan when surfer is injuried
df_events_results['injuried_flag'] = df_events_results['injuried_flag'].fillna(False)

In [ ]:
# create event position result

In [ ]:
df_events_results

,athlete_name,season_year,event_order,points_earned,event_name,event_country,event_wave_side,event_wave_type,event_wave_size,points_earned_update,injuried_flag
0,bobby martinez,2011,1,500.0,quiksilver pro gold coast,australia,right,point,5,265.0,False
1,dane reynolds,2011,1,500.0,quiksilver pro gold coast,australia,right,point,5,265.0,True
2,patrick gudauskas,2013,1,500.0,quiksilver pro gold coast,australia,right,point,5,265.0,False
3,mikey wright,2016,1,1750.0,quiksilver pro gold coast,australia,right,point,5,1330.0,False
4,mikey wright,2017,1,1750.0,quiksilver pro gold coast,australia,right,point,5,1330.0,False
...,...,...,...,...,...,...,...,...,...,...,...
4740,jesse mendes,2019,11,3320.0,billabong pipe masters,hawaii,both-left,barrel,9,3320.0,False
4741,michel bourez,2019,11,4745.0,billabong pipe masters,hawaii,both-left,barrel,9,4745.0,False
4742,griffin colapinto,2019,11,6085.0,billabong pipe masters,hawaii,both-left,barrel,9,6085.0,False
4743,deivid silva,2019,11,1330.0,billabong pipe masters,hawaii,both-left,barrel,9,1330.0,False
